In [1]:
import pandas as pd
import twitter as tt
import tweepy
pd.options.display.max_colwidth=100

In [2]:
import yaml

In [3]:
with open("access_config.yaml") as f:
    access_dict = yaml.safe_load(f)

In [4]:
access_dict

{'api_key': 'TtbfeDYQMKdU3BZQ5kTvChdPm',
 'api_key_secret': 'quP9wjT8PRui2Usz0sEr2kgB56aqW46JaYxJ6hjLAamRQ18qsu',
 'access_key': '1390221257384333312-mR4vfdu1EZycWXJbUR0gVmw3I187ZN',
 'access_key_secret': 'q0sJckhVsKxJy8OC3AfFZCTMzYTcBMigaS2bA5lfeLMz7'}

In [5]:
# authentication
api_key = access_dict['api_key']
api_key_secret = access_dict['api_key']
access_key = access_dict['access_key']
access_key_secret = access_dict['access_key_secret']

In [6]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_key, access_key_secret)

In [7]:
api = tweepy.API(auth)

In [8]:
public_tweets = api.home_timeline()

Unauthorized: 401 Unauthorized
32 - Could not authenticate you.